# DATA 2060 - Fall 2024 - Final Project - Team YYDS

## Overview of Gaussian Naive Bayes classifier

The **Gaussian Naive Bayes (GNB)** classifier is a variant of the *Naive Bayes* algorithm designed for classification problems where the features are continuous rather than binary.
It assumes that the features for each class are **independent** and **follow a Gaussian (normal) distribution**.
These strong assumptions simplify computations, particularly in high-dimensional data scenarios.

This simplicity and efficiency make GNB computationally attractive.
However, real-world datasets often violate the assumptions of feature independence and Gaussian distribution, which can lead to suboptimal performance.
Furthermore, GNB is sensitive to outliers, as they can disproportionately affect the mean and variance estimates of the Gaussian distribution.

In summary, the Gaussian Naive Bayes classifier trades off some predictive accuracy for computational efficiency by relying on strong assumptions about the data.
It is best suited for tasks where efficiency and scalability are prioritized over maximum predictive performance.

### **Representation**
Suppose we have a dataset with $n$ samples, containing $d$ continuous features and a categorical target of $s$ classes.
We will view the features as a $d$-dimensional continuous random vector $X=(X_1,\ldots,X_d)$ and the target as a discrete random variable $Y$ taking value in $\{1,\ldots,s\}$.
We want to find a classifier $h\colon\mathcal{X}\to\mathcal{Y}$, where $\mathcal{X}=\mathbb{R}^d$ and $\mathcal{Y}=\{1,\ldots,s\}$, by estimating the conditional distribution of $Y$ given data $\left(X^{(1)}=x^{(1)},\ldots,X^{(n)}=x^{(n)}\right)$.

#### *prior*
The prior class distribution is given by $p_c=P(Y=c)$ for $c=1,\ldots,s$.

#### *likelihood*
By our assumption, the feature vector $X$ follows a $d$-dimensional jointly normal distribution with its components mutually independent.
The ***probability density function (PDF)*** of a univariate normal random variable, parametrized by its mean $\mu$ and variance $\sigma^2$, is given by
$$f(x)=\frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}.$$
The joint density function of independent normal random variables can then be obtained by simply taking the product of their individual densities:
$$f(x)=f(x_1,\ldots,x_d)=\prod_{k=1}^d f(x_k).$$
Let $X_{c,k}$ denote the normal random variable that represents the $k$-th feature of a sample in class $c$.
Let the parameter tuple $\left(\mu_{c,k},\sigma_{c,k}^2\right)$ denote the mean and variance of $X_{c,k}$.
The (conditional) likelihood of our features can be written as the conditional PDF
$$f_{X|Y}(x|c)=f_{X|Y}(x_1,\ldots,x_d|Y=c)=\prod_{k=1}^d f_{X_k|Y}(x_k|Y=c),$$
where
$$f_{X_k|Y}(x_k|c)=\frac{1}{\sqrt{2\pi\sigma_{c,k}^2}}\exp\left\{-\frac{(x-\mu_{c,k})^2}{2\sigma_{c,k}^2}\right\}$$
for $c=1,\ldots,s$.

#### *joint distribution*
The joint distribution of features and the target cannot be written as a single density.
However, we can write one PDF for each of the values $Y$ can take, i.e., for each $c=1,\ldots,s$ the joint density is given by
$$f_{X,Y}(x,c)=P(Y=c)\cdot f_{X|Y}(x|Y=c).$$

#### *marginal distribution*
The marginal distribution of $X$ is given by summing the class-wise distributions over all values that $Y$ can take, i.e.
$$f_X(x)=\sum_{c=1}^s f_{X,Y}(x,c).$$

#### *posterior*
The posterior of $Y$ conditioned on $X=x$ is given by
$$P(Y=c|X=x)=\frac{f_{X,Y}(x,c)}{f_X(x)}=\frac{P(Y=c)\cdot f_{X|Y}(x|Y=c)}{f_X(x)}$$
for $c=1,\ldots,s$ and $x\in\mathbb{R}^d$.
Note that this representation resembles the ***Bayes' theorem***.

### **Loss**
We can define a cross entropy loss since we are predicting a probability for each class.
For a particular sample $i$, suppose we have predicted $\hat{p}^{(i)}=\left(\hat{p}_1^{(i)},\ldots,\hat{p}_s^{(i)}\right)$ and the true label is $y_i$.
Then its loss is given by $L_i=-\log\hat{p}_{y_i}^{(i)}$.
The overall loss on the whole dataset is then given by
$$L=\frac{1}{n}\sum_{i=1}^n L_i=-\frac{1}{n}\sum_{i=1}^n\log\hat{p}_{y_i}^{(i)}.$$
However, since a loss is not necessary for training a generative model like GNB classifier, this loss serves for an alternative evaluation metric.

### **Optimizer**
The key technique for finding the best classifier is called ***maximum likelihood estimation (MLE)***, which we will use in two situations.

#### *prior*
The prior class distribution $p_c=P(Y=c)$ can be estimated with the empirical class distribution by computing
$$\hat{p}_c=\hat{P}(Y=c)=\frac{n_c}{n}=\frac{1}{n}\sum_{i=1}^n 1_{\{y_i=c\}}$$
for $c=1,\ldots,s$.
Here $n_c$ denotes the number of samples in class $c$.

#### *likelihood*
To estimate the likelihood for each feature given a class, it is sufficient to estimate the mean and variance for each feature and each class, i.e. we will estimate
$$\hat{\mu}_{c,k}=\bar{x}_{c,k}=\frac{1}{n_c}\sum_{i=1}^{n_c}x_{c,k}^{(i)}$$
and
$$\hat{\sigma}_{c,k}^2=S_{c,k}^2=\frac{1}{n_c}\sum_{i=1}^{n_c}\left(x_{c,k}^{(i)}-\bar{x}_{c,k}\right)^2$$
for $c=1,\ldots,s$ and $k=1,\ldots,d$.
Note that there are $2sd$ parameters in total to be estimated, and that we ignore the degree of freedom (dof) in a machine learning context, so the denominator in computing the variance is $n_c$ instead of $n_c-1$.

### **Algorithm Summary**
Suppose we are given a dataset $\left\{\left(x^{(i)},y^{(i)}\right)\right\}_{i=1}^n$, where $x^{(i)}=\left(x_1^{(i)},\ldots,x_d^{(i)}\right)$. The algorithm consists of a training process and a predicting process.

#### *training*
We train the model by estimating all the parameters we need for making predictions.
1. Estimate the prior: $\hat{p}_c$ for $c=1,\ldots,s$.
2. Estimate mean and variance for the likelihood: $\hat{\mu}_{c,k}$ and $\hat{\sigma}_{c,k}^2$ for $c=1,\ldots,s$ and $k=1,\ldots,d$.

#### *predicting*
We use our estimated (thus known and fixed) parameters to make predictions.
Given one sample of data $x=(x_1,\ldots,x_d)$, we will follow the steps as shown below while striving to replace `for` loops with vectorized computations.
1. For $c=1,\ldots,s$ and $k=1,\ldots,d$, compute the log-likelihood:
$$\log f_{X_k|Y}(x_k|c)=-\frac{1}{2}\log\left(2\pi\hat{\sigma}_{c,k}^2\right)-\frac{(x-\hat{\mu}_{c,k})^2}{2\hat{\sigma}_{c,k}^2}$$
2. Convert the prior into log space.
3. For $c=1,\ldots,s$, sum up the log-likelihood over $k$ with the log-prior added to it (denoted by $l_c$):
$$l_c=\log\hat{p}_c+\sum_{k=1}^d\log f_{X_k|Y}(x_k|c)=\log\left(\hat{p}_c\cdot\prod_{k=1}^d f_{X_k|Y}(x_k|c)\right)=\log\left(\hat{p}_c\cdot f_{X|Y}(x|c)\right)=\log f_{X,Y}(x,c).$$
4. Predict the class with the highest value of $l_c$:
$$\hat{y}=\argmax_{c=1,\ldots,s}l_c.$$
Note that step 4 works because
- $f_X(x)$ is a normalizing constant independent of $c$, so the posterior $P(Y=c|X=x)$ is proportional to $f_{X,Y}(x,c)$.
- the log function is monotonically increasing, so applying it to an array does not change the maximizer.

#### *loss*
In the case where we want to compute the loss from predicting one sample $(x,y)$, we will follow the steps as shown below.
1. For $c=1,\ldots,s$, convert $l_c$ back out of log space by exponentiating it:
$$f_{X,Y}(x,c)=e^{l_c}.$$
2. Compute the marginal likelihood of $x$:
$$f_X(x)=\sum_{c=1}^s f_{X,Y}(x,c).$$
3. Compute the predicted probability for the true class $y$:
$$\hat{P}(Y=y|X=x)=\frac{f_{X,Y}(x,y)}{f_X(x)}.$$
4. Apply the cross-entropy loss function:
$$L=-\log\hat{P}(Y=y|X=x).$$
The loss from predicting the whole dataset can then be obtained by simply taking the mean over the losses from all samples computed the same way as above.

## Model

In [1]:
import numpy as np

def normal_log_pdf(x: int | float | np.ndarray,
                   mu: int | float | np.ndarray,
                   sig2: int | float | np.ndarray):
    """
    Computes the natural log of the probability density function of
    a normal distribution with mean `mu` and variance `sig2` evaluated at `x`;
    Note: can be applied to a NumPy array, in which case the shapes of
    all arguments are expected to conform or be able to broadcast

    Args:
        `x`: the value(s) at which the pdf is evaluated
        `mu`: mean of the normal distribution
        `sig2`: variance of the normal distribution

    Returns: log-likelihood of observing `x` under the given distribution
    """
    return (-np.log(2*np.pi*sig2) - (x-mu)**2/sig2) / 2

class GaussianNaiveBayes:
    def __init__(self, var_smoothing: float=1e-9):
        """
        Initializes the Gaussian Naive Bayes classifier
        """
        self.var_smoothing = var_smoothing
        self.n_classes = None
        self.priors = None
        self.mean = None
        self.var = None

    def train(self, X: np.ndarray, y: np.ndarray):
        """
        Trains the classifier by estimating and storing the following attributes:
        - class priors
        - mean for each class and each feature
        - variance for each class and each feature

        Args:
            `X`: an array of shape (n_samples, n_features) containing feature values
            `y`: an array of shape (n_samples,) containing true class labels
        
        Note:
            Argument `y` is expected to be prepared such that
            if there are n classes in total,
            then `y` contains labels 0, 1, 2, ..., n-1.
        """
        n_samples, n_features = X.shape
        labels, counts = np.unique(y, return_counts=True)
        self.n_classes = labels.size
        self.priors = counts / n_samples # (n_classes,)
        self.mean = np.zeros((self.n_classes, n_features))
        self.var = np.zeros_like(self.mean) # (n_classes, n_features)
        for label in labels:
            mask = (y==label) # (n_samples,)
            x = X[mask] # (local_n_samples, n_features)
            assert x.shape[0] == counts[label]
            self.mean[label] = x.mean(axis=0) # (n_features,)
            self.var[label] = x.var(axis=0) # (n_features,)
        max_var = self.var.max()
        self.var += self.var_smoothing * max(1, max_var) # (n_classes, n_features)

    def display_estimated_params(self):
        """
        Displays estimated parameters of the model
        """
        print("Estimated parameters:")
        print(f"priors\n{self.priors}")
        print(f"means\n{self.mean}")
        print(f"variances\n{self.var}")

    def predict(self, X: np.ndarray, return_proba: bool=False):
        """
        Predicts labels of the input feature

        If `return_proba` is True, also returns the predicted probabilities while

        Args:
            `X`: an array of shape (n_samples, n_features) containing feature values
            `return_proba`: defaults to be False

        Returns:
            `y_pred`: an array of shape (n_samples,) containing predicted class labels
            `proba`: an array of shape (n_samples, n_classes) containing predicted probabilities
        """
        n_samples, n_features = X.shape
        X_aug = X.reshape(n_samples, 1, n_features)
        mean_aug = self.mean.reshape(1, self.n_classes, n_features)
        var_aug = self.var.reshape(1, self.n_classes, n_features)
        log_priors = np.log(self.priors) # (n_classes,)
        log_L = normal_log_pdf(X_aug, mean_aug, var_aug) # (n_samples, n_classes, n_features)
        log_joint = log_priors + log_L.sum(axis=2) # (n_samples, n_classes)
        y_pred = log_joint.argmax(axis=1) # (n_samples,)
        if return_proba:
            proba = self._get_proba(log_joint) # (n_samples, n_classes)
            return y_pred, proba
        return y_pred

    def _get_proba(self, log_joint):
        """
        Calculates predicted probabilities while storing `log_proba`

        Returns:
            `proba`: an array of shape (n_samples, n_classes) containing predicted probabilities
        """
        C = log_joint.max()
        joint_proba = np.exp(log_joint - C) # (n_samples, n_classes)
        marginal = joint_proba.sum(axis=1) # (n_samples,)
        proba = joint_proba / marginal.reshape(-1, 1) # (n_samples, n_classes)
        return proba

    def accuracy(self, X: np.ndarray, y: np.ndarray,
                 return_pred: bool=False,
                 return_proba: bool=False,
                 return_loss: bool=False,):
        """
        Calculates accuracy of prediction

        If `return_pred` is True, also returns the predicted labels

        If `return_proba` is True, also returns the predicted probabilities

        If `return_loss` is True, also returns the cross entropy loss on the given dataset

        Args:
            `X`: an array of shape (n_samples, n_features) containing feature values
            `y`: an array of shape (n_samples,) containing true class labels
            `return_pred`: defaults to be False
            `return_proba`: defaults to be False
            `return_loss`: defaults to be False

        Returns:
            `accuracy`: a scalar that represents the proportion of points that are
            predicted correctly
            `y_pred`: an array of shape (n_samples,) containing predicted class labels
            `proba`:  an array of shape (n_samples, n_classes) containing predicted probabilities
            `loss`: a scalar that represents the cross entropy loss on the given dataset
        """
        if not (return_proba or return_loss):
            y_pred = self.predict(X)
        else:
            y_pred, proba = self.predict(X, return_proba=True)
        accuracy = (y_pred==y).mean()
        outputs = [accuracy.item()]
        if return_pred:
            outputs.append(y_pred)
        if return_proba:
            outputs.append(proba)
        if return_loss:
            loss = self._loss(y, proba)
            outputs.append(loss)
        return tuple(outputs)

    def _loss(self, y_true: np.ndarray, proba: np.ndarray):
        """
        Calculates the cross entropy loss given predicted probabilities and true labels

        Args:
            `y_true`: an array of shape (n_samples,) containing true class labels
            `proba`: an array of shape (n_samples, n_classes) containing predicted probabilities

        Returns:
            `loss`: a scalar that represents the cross entropy loss
        """
        n_samples = y_true.size
        log_proba = np.log(proba)
        loss = (-log_proba[np.arange(n_samples), y_true]).mean()
        return loss.item()

## Check Model

In [2]:
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# fix random seed and state
np.random.seed(42)
RANDOM_STATE = 42

# instantiate our model and sklearn model
model = GaussianNaiveBayes()
clf = GaussianNB()

Test helper function

In [3]:
# test normal_log_pdf
pdf_test_x = np.array([-1, 0, 1])
pdf_test_mu = np.array([1, 2, 3])
pdf_test_var = np.array([1, 4, 9])
func_output = normal_log_pdf(pdf_test_x, pdf_test_mu, pdf_test_var)
scipy_output = np.log(norm.pdf(pdf_test_x, pdf_test_mu, np.sqrt(pdf_test_var)))
assert np.isclose(func_output, scipy_output).all()

Test model with a simple dataset

In [4]:
# test simple case
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([0, 0, 0, 1, 1, 1])
X_pred = np.array([[-0.8, -1]])
# train
model.train(X, y)
clf.fit(X, y)
assert np.isclose(model.priors, clf.class_prior_).all()
assert np.isclose(model.mean, clf.theta_).all()
assert np.isclose(model.var, clf.var_).all()
# predict
model_y_pred = model.predict(X_pred)
clf_y_pred = clf.predict(X_pred)
assert (model_y_pred == clf_y_pred).all()

Test with an edge case where
- there are two classes
- values within each class and each feature are identical

In [5]:
# test edge case
X = np.array([[1, -1], [1, -1], [-1, 1], [-1, 1]])
y = np.array([0, 0, 1, 1])
X_pred = np.array([[-0.8, -1]])
# train
model.train(X, y)
clf.fit(X, y)
assert np.isclose(model.priors, clf.class_prior_).all()
assert np.isclose(model.mean, clf.theta_).all()
assert np.isclose(model.var, clf.var_).all()
# predict
model_y_pred = model.predict(X_pred)
clf_y_pred = clf.predict(X_pred)
assert (model_y_pred == clf_y_pred).all()

Test with an extreme edge case where
- there is only one class
- values within each feature are identical

PS: the `sklearn` version reports `RunTimeWarning` here

In [6]:
# test extreme edge case
X = np.array([[1, -1], [1, -1]])
y = np.array([0, 0])
X_pred = np.array([[-0.8, -1]])
# train
model.train(X, y)
clf.fit(X, y)
assert np.isclose(model.priors, clf.class_prior_).all()
assert np.isclose(model.mean, clf.theta_).all()
assert np.isclose(model.var, clf.var_).all()
# predict
model_y_pred = model.predict(X_pred)
clf_y_pred = clf.predict(X_pred)
assert (model_y_pred == clf_y_pred).all()

/opt/anaconda3/envs/data2060/lib/python3.12/site-packages/sklearn/naive_bayes.py:510: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/opt/anaconda3/envs/data2060/lib/python3.12/site-packages/sklearn/naive_bayes.py:511: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/opt/anaconda3/envs/data2060/lib/python3.12/site-packages/sklearn/naive_bayes.py:511: RuntimeWarning: invalid value encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)


Test performance on a generated large dataset that satisfies assumptions of Gaussian Naive Bayes

In [7]:
# test with larger dataset
def create_dataset(n_samples: int, prior: np.ndarray, params: np.ndarray):
    """
    Generates a dataset that satisfies the assumptions of Gaussian Naive Bayes

    Args:
        `n_samples`: an integer indicating the size (# points) of the dataset
        `prior`: an array of shape (n_classes,) containing the class prior
        Note: `prior` should satisfy
            (a) each entry is positive
            (b) all entries sum up to 1
        `params`: an array of shape (2, n_classes, n_features) containing
        parameters of in-class normal distribution for each class and each feature
        Note: the 2 entries of axis 0 of `params` respectively represent
        mean and varaince of the normal distribution; all variances (represented by
        `params[1]`) are expected to be positive

    Returns:
        `X`: an array of shape (n_samples, n_features) containing feature values
        `y`: an array of shape (n_samples,) containing true class labels
    """
    assert prior.ndim == 1
    assert (prior > 0).all()
    assert prior.sum() == 1
    assert params.ndim == 3
    n, n_class, n_features = params.shape
    assert n_class == prior.shape[0]
    assert n == 2
    assert (params[1] > 0).all()
    label_counts = np.random.multinomial(n_samples, pvals=prior)
    X_list, y_list = [], []
    for label, count in enumerate(label_counts):
        y_list.append(label * np.ones(count, dtype=int))
        mean = params[0, label]
        var = params[1, label]
        std = np.sqrt(var)
        X_list.append(np.random.normal(mean, std, (count, n_features)))
    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    X[:] = X[indices]
    y[:] = y[indices]
    return X, y

# generate dataset and split
test_prior = np.array([0.2, 0.5, 0.3])
test_mean = np.random.uniform(-10.0, 10.0, (3, 4))
test_var = np.random.gamma(10.0, size=(3, 4))
print("Parameters for generating dataset:")
print(f"priors\n{test_prior}\nmeans\n{test_mean}\nvariances\n{test_var}")
test_params = np.stack((test_mean, test_var), axis=0)
X, y = create_dataset(int(1.25e5), test_prior, test_params)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)

# our model
model.train(X_train, y_train)
model.display_estimated_params()
acc, y_pred, proba, loss = model.accuracy(
    X_test, y_test, return_pred=True, return_proba=True, return_loss=True)

# sklearn model
clf.fit(X_train, y_train)
clf_y_pred = clf.predict(X_test)
clf_proba = clf.predict_proba(X_test)
clf_acc = clf.score(X_test, y_test)

# test train
assert np.isclose(model.priors, clf.class_prior_).all()
assert np.isclose(model.mean, clf.theta_).all()
assert np.isclose(model.var, clf.var_).all()

# test predict
assert (y_pred == clf_y_pred).all()
assert np.isclose(proba, clf_proba).all()

# print accuracies
print(f"Test accuracy by our model: {acc}")
print(f"Test accuracy by sklearn model: {clf_acc}")

Parameters for generating dataset:
priors
[0.2 0.5 0.3]
means
[[-2.50919762  9.01428613  4.63987884  1.97316968]
 [-6.87962719 -6.88010959 -8.83832776  7.32352292]
 [ 2.02230023  4.16145156 -9.58831011  9.39819704]]
variances
[[ 8.27924758 11.45358077 10.43864194  4.85481822]
 [ 6.84721222 10.67698813 14.97710084  8.98155417]
 [ 8.07097436 10.01565654  7.9168794   8.78782004]]
Estimated parameters:
priors
[0.19969 0.50148 0.29883]
means
[[-2.51465312  9.02480353  4.68318818  1.95030105]
 [-6.87359249 -6.88080238 -8.83469895  7.30144095]
 [ 2.02503691  4.15016076 -9.61565662  9.42510813]]
variances
[[ 8.44881989 11.32674187 10.47666018  4.81761254]
 [ 6.86000106 10.70073827 14.89365958  8.96458058]
 [ 8.03148477 10.0843887   7.90376508  8.89021161]]
Test accuracy by our model: 0.99292
Test accuracy by sklearn model: 0.99292


Run both models on the wine dataset

In [8]:
import pandas as pd

# prepare data
column_names = [
    'class',
    'Alcohol',
    'Malicacid',
    'Ash',
    'Alcalinity_of_ash',
    'Magnesium',
    'Total_phenols',
    'Flavanoids',
    'Nonflavanoid_phenols',
    'Proanthocyanins',
    'Color_intensity',
    'Hue',
    '0D280_0D315_of_diluted_wines',
    'Proline',
]
df = pd.read_csv('../data/Wine.csv', names=column_names, header=None)
y = df['class'].to_numpy()
y -= 1
X = df.loc[:, df.columns != 'class'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)

# our model
model.train(X_train, y_train)
clf.fit(X_train, y_train)
acc, y_pred = model.accuracy(
    X_test, y_test, return_pred=True)

# sklearn model
clf_y_pred = clf.predict(X_test)
clf_acc = clf.score(X_test, y_test)

# test prediction
assert (y_pred == clf_y_pred).all()

# print accuracies
print(f"Test accuracy by our model: {acc}")
print(f"Test accuracy by sklearn model: {clf_acc}")

Test accuracy by our model: 1.0
Test accuracy by sklearn model: 1.0
